In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
from collections import defaultdict
from empyrical import sharpe_ratio, sortino_ratio, calmar_ratio, max_drawdown, annual_return

In [ ]:
multivariate_data = ['lstm_seed_42_2.pickle', 'conv_seed_42_2.pickle',
                     'slp_seed_42_2.pickle', 'mlp_seed_42_2.pickle']

In [ ]:
reg_data = ['lstm_seed_42_2.pickle', 'lstm_seed_42_2_reg_1.pickle',
            'lstm_seed_42_2_reg_2.pickle', 'lstm_seed_42_2_reg_3.pickle']

In [ ]:
univariate_data = ['lstm_seed_42_univariate.pickle', 'conv_seed_42_univariate.pickle']

In [ ]:
m_results = {}
for path in multivariate_data:
    with open(os.path.join('results', path), 'rb') as f:
        results = pickle.load(f)
    model_name = path.split('_')[0]
    m_results[model_name] = results

In [ ]:
u_results = {}
for path in univariate_data:
    with open(os.path.join('results', path), 'rb') as f:
        results = pickle.load(f)
    model_name = path.split('_')[0]
    u_results[model_name] = results

In [ ]:
r_results = {}
for j, path in enumerate(reg_data):
    with open(os.path.join('results', path), 'rb') as f:
        results = pickle.load(f)
    model_name = path.split('_')[0] +'_{}'.format(j)
    r_results[model_name] = results

In [ ]:
val_delta = pd.Timedelta('365days')
test_delta = pd.Timedelta('365days')
basis_points = [0, 1, 5, 10]
target_vol = 0.15
metrics = {"Sharpe Ratio": sharpe_ratio,
           "Sortino Ratio": sortino_ratio,
           "Calmar Ratio": calmar_ratio,
           "Max Drawdown": max_drawdown,
           "Annualized Return": annual_return}

In [ ]:
def unpack_data(results, ts, fold, c):
    preds = results[ts][fold]['preds']
    returns = results[ts][fold]['returns']
    vols = results[ts][fold]['vols']
    
    vols = vols * 252**0.5
    T = target_vol*np.abs(np.diff(preds/(vols+1e-12), prepend=0.0, axis=0))
    captured_returns = preds*returns
    
    R = captured_returns - 1e-4*c*T
    R = np.mean(R, axis=1)
    
    return R, preds, vols, returns


def ts_to_string(ts):
    return ts.to_numpy().astype(str).split('T')[0]


def plot_results(results):
    models = list(results.keys())
    timesteps = list(results[models[0]].keys())
    
    for c in basis_points:
        for current_metric in metrics.keys():
            metrics_val, metrics_test = defaultdict(list), defaultdict(list)
            captured_val, captured_test = defaultdict(list), defaultdict(list)
            preds_val, preds_test = defaultdict(list), defaultdict(list)
            vols_val, vols_test = [], []
            returns_val, returns_test = [], []
            val_timesteps, test_timesteps = [], []
            for ts in timesteps:
                for model in models:
                    R_val, preds_val_, vols_val_, returns_val_ = unpack_data(results[model], ts, 'val', c)
                    R_test, preds_test_, vols_test_, returns_test_ = unpack_data(results[model], ts, 'test', c)
                    
                    preds_val[model].append(preds_val_)
                    preds_test[model].append(preds_test_)
                    
                    captured_val[model].append(R_val)
                    captured_test[model].append(R_test)
                    
                    metrics_val[model].append(metrics[current_metric](R_val))
                    metrics_test[model].append(metrics[current_metric](R_test))
                    
                val_timesteps.append(ts_to_string(ts+val_delta))
                test_timesteps.append(ts_to_string(ts+val_delta+test_delta))
                
                returns_val.append(returns_val_)
                returns_test.append(returns_test_)
                vols_val.append(vols_val_)
                vols_test.append(vols_test_)
                
            fig, ax = plt.subplots(1, 2, figsize=(20, 5), sharey=True)
            ax[0].set_title('Metric: {}, fold: {}, basis point: {}'.format(current_metric, 'val', c))
            ax[1].set_title('Metric: {}, fold: {}, basis point: {}'.format(current_metric, 'test', c))
            
            for model in metrics_val.keys():
                ax[0].plot(metrics_val[model], label=model, marker='o')
            ax[0].set_xticks(range(len(val_timesteps)), val_timesteps, rotation=45)
            ax[0].set_ylabel(current_metric)
                
            for model in metrics_test.keys():
                ax[1].plot(metrics_test[model], label=model, marker='o')
            ax[1].set_xticks(range(len(test_timesteps)), test_timesteps, rotation=45)
            ax[1].set_ylabel(current_metric)
                
            
            ax[0].legend()
            ax[1].legend()
            plt.show()
            
            fig, ax = plt.subplots(1, 2, figsize=(20, 5), sharey=True)
            
            ax[0].set_title('Average Metric: {}, fold: {}, basis point: {}'.format(current_metric, 'val', c))
            ax[1].set_title('Average Metric: {}, fold: {}, basis point: {}'.format(current_metric, 'test', c))
            
            ax[0].bar(range(len(models)), [np.mean(metrics_val[model]) for model in metrics_val.keys()])
            ax[0].set_xticks(range(len(models)), models)
            ax[0].set_ylabel(current_metric)
            
            ax[1].bar(range(len(models)), [np.mean(metrics_test[model]) for model in metrics_test.keys()])
            ax[1].set_xticks(range(len(models)), models)
            ax[1].set_ylabel(current_metric)
            plt.show()
            
            fig, ax = plt.subplots(1, 2, figsize=(20, 5), sharey=True)
            ax[0].set_title('Model Ensemble, Metric: {}, fold: {}, basis point: {}'.format(current_metric, 'val', c))
            ax[1].set_title('Model Ensemble, Metric: {}, fold: {}, basis point: {}'.format(current_metric, 'test', c))
            
            m_val, m_test = [], []
            for i in range(len(preds_val[models[0]])):
                p_val, p_test = [], []

                for model in preds_val.keys():
                    p_val.append(preds_val[model][i][np.newaxis, ...])
                    p_test.append(preds_test[model][i][np.newaxis, ...])
                    
                p_val = np.concatenate(p_val)
                p_test = np.concatenate(p_test)
                p_val = np.mean(p_val, axis=0)
                p_test = np.mean(p_test, axis=0)

                T_val = target_vol*np.abs(np.diff(p_val/(vols_val[i]+1e-12), prepend=0.0, axis=0))
                T_test = target_vol*np.abs(np.diff(p_test/(vols_test[i]+1e-12), prepend=0.0, axis=0))

                c_val = returns_val[i]*p_val - 1e-4*c*T_val
                c_test = returns_test[i]*p_test - 1e-4*c*T_test

                c_val = np.mean(c_val, axis=1)
                c_test = np.mean(c_test, axis=1)

                m_val_ = metrics[current_metric](c_val)
                m_test_ = metrics[current_metric](c_test)
                m_val.append(m_val_)
                m_test.append(m_test_)

            ax[0].plot(m_val, marker='o')
            ax[1].plot(m_test, marker='o')
            ax[0].set_ylabel(current_metric)
            ax[1].set_ylabel(current_metric)
            ax[0].set_xticks(range(len(val_timesteps)), val_timesteps, rotation=45)
            ax[1].set_xticks(range(len(test_timesteps)), test_timesteps, rotation=45)
            
        fig, ax = plt.subplots(1, 2, figsize=(20, 5), sharey=True)
        
        ax[0].set_title('Cumulative Returns, fold: val')
        for model in captured_val.keys():
            rets = np.concatenate(captured_val[model])
            cr = 1 + rets
            cr = np.cumprod(cr)
            ax[0].plot(cr, label=model)
        
        ax[1].set_title('Cumulative Returns, fold: test')
        for model in captured_test.keys():
            rets = np.concatenate(captured_test[model])
            cr = 1 + rets
            cr = np.cumprod(cr)
            ax[1].plot(cr, label=model)
        ax[0].set_xlabel('number of days')
        ax[0].set_ylabel('Cumulative Returns')
        ax[0].legend()
        ax[1].set_xlabel('number of days')
        ax[1].set_ylabel('Cumulative Returns')
        ax[1].legend()
        plt.show()
        

In [ ]:
plot_results(m_results)

In [ ]:
plot_results(u_results)

In [ ]:
plot_results(r_results)